### Web Scraping
This is a web scraper I built using Selenium and Beautiful Soup to scrape information from Linkedin. From what I read online, Linkedin's API is pretty restrictive in terms of the amount of information it allows you to attain. Hence, I opted to scrape the information myself manually. Anyways, I wanted to practice building a scraper too :) 

I collected the following information:
- Job title
- Company name
- Country
- When the job opening was posted
- Number of applicants
- Job hyperlink
- Job description

Scraping Linkedin has its challenges too. Linkedin, like many other websites, uses an *infinity scroll*, meaning instead of clicking a button to navigate to the next page, you had to scroll to the bottom of the current page and wait for the next page to *append* to the existing page. This was a slight inconvenience but it was overcame with the *scroll* function. What could not be overcome, however, was Linkedin's anti-scraping feature, which allowed a user to only view the first 1000 jobs of a search. But it was alright for me as I felt that 1000 results per search was sufficient.

#### Import libraries

In [4]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from time import sleep
from selenium.webdriver.common.keys import Keys
# chromedriver.exe must be in the same folder

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

from nltk.tokenize import RegexpTokenizer
retokenizer = RegexpTokenizer(r'[\w@]+')
import pandas as pd

In [ ]:
## ONLY RUN IF CHROMEDRIVER IS OUTDATED ##
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

In [9]:
def scroll(browser, timeout):
    # Get scroll height
    last_height = browser.execute_script('return document.body.scrollHeight')
    
    while True:
        # Scroll down to the bottom
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        
        # Wait for page to load
        sleep(timeout)
        
        try:
            seemorejobs_button = browser.find_element_by_xpath('//*[@id="main-content"]/div/section/button').click()
            sleep(timeout)
        except:
            pass

        
        # Calculate new scroll height and compare w last scroll height
        new_height = browser.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
            
        last_height = new_height

# Scrape Linkedin | Create CSV File

In [6]:
def linkedin_scraper(jobs):
    for job in jobs:
        search_this_job = job
        url = 'https://www.linkedin.com/jobs'
        job_search_bar_xpath = '/html/body/main/section[1]/section/div[2]/section[2]/form/section[1]/input'
        show_more_button_xpath = '/html/body/main/section[1]/section[3]/div/section/button[1]'

        # Access linkedin
        browser = webdriver.Chrome()
        browser.get(url)
        sleep(1)

        job_search_bar = browser.find_element_by_xpath(job_search_bar_xpath)
        job_search_bar.click()
        job_search_bar.send_keys(search_this_job)
        job_search_bar.send_keys(Keys.ENTER)

        sleep(1)

        # Scroll to bottom of infinity page
        scroll(browser, 3) # second parameter refers to number of sec the webdriver sleeps for, incr if your machine is slow

        # Soup main page and filter out the hyperlinks
        main_soup = soup(browser.page_source, 'lxml')
        containers =  main_soup.findAll('a', {'class': 'result-card__full-card-link'})
        links = [i['href'] for i in containers]
        print(len(links))

        # Write results to csv file
        filename = 'linkedin_' + search_this_job + '.csv'
        f = open(filename, 'w',  encoding="utf-8")
        headers = 'Job Title,Company Name,Country,When Posted,No. of Applicants,Hyperlink,Job Description\n'
        f.write(headers)

        #### Souping each page ####

        for num in range(len(links)):
            print('now at job number: ' + str(num + 1) + ' of ' + str(len(links)))
            try:
                try:
                    current_link = links[num]
                    browser.get(current_link)
                    current_soup = soup(browser.page_source, 'lxml')
                except:
                    continue

                # Press Show More dropdown button
                try:
                    search_for_showmore_button = browser.find_element_by_xpath(show_more_button_xpath).click()
                except:
                    pass

                # Job Hyperlink
                job_hyperlink = links[num]
                print(job_hyperlink)

                # Job title
                try:
                    job_title = current_soup.find('h1', {'class': 'topcard__title'}).text.replace(',', '|')
                    print(job_title)
                except:
                    pass

                # Company name
                try:
                    coy_name = current_soup.find('a', {'data-tracking-control-name': 'public_jobs_topcard_org_name'}).text.replace(',', '|')
                except:
                    coy_name = current_soup.find('span', {'class': 'topcard__flavor'}).text.replace(',', '|')
                print(coy_name)

                # Country
                country = current_soup.find('span', {'class': 'topcard__flavor topcard__flavor--bullet'}).text.replace(',', '|')
                print(country)

                # Posted how long ago
                try:
                    when_posted = current_soup.find('span', {'class': 'topcard__flavor--metadata posted-time-ago__text'}).text.replace(',', '|')
                except:
                    when_posted = 'nil'
                print(when_posted)

                # Number of applicants
                try:
                    num_applicants = current_soup.find('span', {'class': 'topcard__flavor--metadata topcard__flavor--bullet num-applicants__caption'}).text.replace(',', '|')
                except:
                    num_applicants = current_soup.find('figcaption', {'class': 'num-applicants__caption'}).text
                print(num_applicants)
                print('\n')

                # Job Description BoW
                job_desc_bow = current_soup.find('div', {'class': 'show-more-less-html__markup'})
                if job_desc_bow == None:
                    job_desc_bow = current_soup.find('div', {'class': 'description__text description__text--rich'})
                job_desc_bow = str(job_desc_bow)
                job_desc_bow = job_desc_bow.replace(',', '')

            except:
                print('AN ERROR WITH THIS JOB OCCURRED, CONTINUING TO NEXT JOB')
                job_title = 'error'
                coy_name = 'error'
                country = 'error'
                when_posted = 'error'
                num_applicants = 'error'
                job_hyperlink = 'error'
                tokens_str = 'error'

            f.write(job_title + ',' + coy_name + ',' + country + ',' + when_posted + ',' + num_applicants + ',' + job_hyperlink + ',' + str(job_desc_bow) + '\n')

        f.close()

#### Input job searches to scrape

In [8]:
search_these_jobs = ['cybersecurity', 'business analyst', 'software developer']
linkedin_scraper(search_these_jobs)

991
now at job number: 1 of 991
https://sg.linkedin.com/jobs/view/global-security-director-apac-at-facebook-2216436939?refId=801bdef2-53a1-41f9-ac7a-b6e5fb89a077&position=1&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click
Global Security Director| APAC
Facebook
Singapore| Singapore
3 weeks ago
39 applicants


now at job number: 2 of 991
https://sg.linkedin.com/jobs/view/security-consulting-trainee-at-paypal-2198877052?refId=801bdef2-53a1-41f9-ac7a-b6e5fb89a077&position=2&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click
Security Consulting Trainee
PayPal
Singapore| Singapore
4 days ago
Be among the first 25 applicants


now at job number: 3 of 991
https://sg.linkedin.com/jobs/view/associate-cybersecurity-analyst-at-visa-2166982052?refId=801bdef2-53a1-41f9-ac7a-b6e5fb89a077&position=3&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click
Associate Cybersecurity Analyst
Visa
Singapore| Singapore
3 weeks ago
69 applicants


now at job number: 4 of

https://sg.linkedin.com/jobs/view/lead-cybersecurity-specialist-at-hackertrail-2168476193?refId=801bdef2-53a1-41f9-ac7a-b6e5fb89a077&position=24&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click
Lead Cybersecurity Specialist
HackerTrail
Singapore| Singapore
3 weeks ago
78 applicants


now at job number: 25 of 991
https://sg.linkedin.com/jobs/view/2021-mastercard-launch-associate-specialist-cyber-intelligence-solutions-at-mastercard-2209332790?refId=801bdef2-53a1-41f9-ac7a-b6e5fb89a077&position=25&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click
2021 Mastercard Launch| Associate Specialist| Cyber & Intelligence Solutions
Mastercard
Singapore| Singapore
4 weeks ago
Be among the first 25 applicants


now at job number: 26 of 991
https://sg.linkedin.com/jobs/view/%23sgunitedtraineeships-associate-cybersecurity-consultant-trainee-verbosecurity-at-singapore-institute-of-technology-2214708601?refId=297ebd70-c9ba-4c54-b76d-122bffa8319e&position=1&pageNum=1&trk=pu

https://sg.linkedin.com/jobs/view/expert-or-lead-threat-hunter-analyst-singapore-or-hong-kong-at-network-guard-1981158885?refId=297ebd70-c9ba-4c54-b76d-122bffa8319e&position=22&pageNum=1&trk=public_jobs_job-result-card_result-card_full-click
Expert or Lead Threat Hunter/Analyst [Singapore or Hong Kong]
Network Guard
Singapore
4 days ago
Be among the first 25 applicants


now at job number: 48 of 991
https://sg.linkedin.com/jobs/view/consulting-intern-%E2%80%93-cybersecurity-financial-services-singapore-winter-2020-at-acca-careers-2205258014?refId=297ebd70-c9ba-4c54-b76d-122bffa8319e&position=23&pageNum=1&trk=public_jobs_job-result-card_result-card_full-click
Consulting Intern – Cybersecurity (Financial Services)| Singapore (Winter 2020)
ACCA Careers
Singapore| Singapore
2 weeks ago
Be among the first 25 applicants


now at job number: 49 of 991
https://sg.linkedin.com/jobs/view/lead-cybersecurity-specialist-at-wildlife-reserves-singapore-wrs-2198810606?refId=297ebd70-c9ba-4c54-b76d-122

https://sg.linkedin.com/jobs/view/head-of-infrastructure-security-at-selby-jennings-2150989136?refId=7dff88a4-bf1a-4fd1-a467-1961f92e6c8a&position=21&pageNum=2&trk=public_jobs_job-result-card_result-card_full-click
Head of Infrastructure Security
Selby Jennings
Singapore| Singapore
4 weeks ago
130 applicants


now at job number: 71 of 991
https://sg.linkedin.com/jobs/view/cyberdefence-threat-detection-response-engineer-at-lazada-2184378447?refId=7dff88a4-bf1a-4fd1-a467-1961f92e6c8a&position=22&pageNum=2&trk=public_jobs_job-result-card_result-card_full-click
Cyberdefence Threat Detection & Response Engineer
Lazada
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 72 of 991
https://sg.linkedin.com/jobs/view/information-security-officer-at-atome-2184366060?refId=7dff88a4-bf1a-4fd1-a467-1961f92e6c8a&position=23&pageNum=2&trk=public_jobs_job-result-card_result-card_full-click
Information Security Officer
Atome
Singapore| Singapore
1 week ago
70 applicants

https://sg.linkedin.com/jobs/view/it-security-specialist-at-housing-development-board-2211759017?refId=5f969661-befd-4544-89d7-b8fdf728125e&position=18&pageNum=3&trk=public_jobs_job-result-card_result-card_full-click
IT Security Specialist
Housing & Development Board
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 93 of 991
https://sg.linkedin.com/jobs/view/graduate-recent-cyber-security-consultant-offensive-security-singapore-at-wizlynx-group-1852876723?refId=5f969661-befd-4544-89d7-b8fdf728125e&position=19&pageNum=3&trk=public_jobs_job-result-card_result-card_full-click
Graduate (Recent) Cyber Security Consultant - Offensive Security - Singapore
wizlynx group
Singapore| Singapore
1 week ago
Over 200 applicants


now at job number: 94 of 991
https://sg.linkedin.com/jobs/view/lead-senior-security-consultant-governance-risk-compliance-at-cpf-board-2201644292?refId=5f969661-befd-4544-89d7-b8fdf728125e&position=20&pageNum=3&trk=public_jobs_job-result-

...

https://sg.linkedin.com/jobs/view/software-developer-temenos-at-dow-technologies-and-systems-pte-ltd-2225286310?refId=a36b2f9b-51de-480e-8fa5-697342980d05&position=4&pageNum=36&trk=public_jobs_job-result-card_result-card_full-click
Software Developer - Temenos
DOW TECHNOLOGIES AND SYSTEMS PTE. LTD.
Singapore| Singapore
4 days ago
Be among the first 25 applicants


now at job number: 877 of 968
https://sg.linkedin.com/jobs/view/software-engineer-at-reset-private-limited-2203910721?refId=a36b2f9b-51de-480e-8fa5-697342980d05&position=5&pageNum=36&trk=public_jobs_job-result-card_result-card_full-click
Software Engineer
RESET PRIVATE LIMITED
Singapore| Singapore
2 weeks ago
Be among the first 25 applicants


now at job number: 878 of 968
https://sg.linkedin.com/jobs/view/software-engineer-backend-at-motional-singapore-pte-limited-2207051476?refId=a36b2f9b-51de-480e-8fa5-697342980d05&position=6&pageNum=36&trk=public_jobs_job-result-card_result-card_full-click
Software Engineer (Backend)
MOTI

https://sg.linkedin.com/jobs/view/software-engineer-java-development-at-nityo-infotech-services-pte-ltd-2212028315?refId=e80d85be-dc72-4ab5-9297-1898486fba34&position=2&pageNum=37&trk=public_jobs_job-result-card_result-card_full-click
Software Engineer - Java Development
NITYO INFOTECH SERVICES PTE. LTD.
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 899 of 968
https://sg.linkedin.com/jobs/view/software-developer-net-mvc-at-permanent-personnel-services-pte-ltd-2209834289?refId=e80d85be-dc72-4ab5-9297-1898486fba34&position=3&pageNum=37&trk=public_jobs_job-result-card_result-card_full-click
Software Developer (.net MVC)
PERMANENT PERSONNEL SERVICES PTE LTD
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 900 of 968
https://sg.linkedin.com/jobs/view/mobile-software-developer-at-cartrack-2217361514?refId=e80d85be-dc72-4ab5-9297-1898486fba34&position=4&pageNum=37&trk=public_jobs_job-result-card_result-card_full-clic

https://sg.linkedin.com/jobs/view/software-developer-at-thaibev-group-2236640619?refId=e80d85be-dc72-4ab5-9297-1898486fba34&position=24&pageNum=37&trk=public_jobs_job-result-card_result-card_full-click
Software Developer
ThaiBev Group
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 921 of 968
https://sg.linkedin.com/jobs/view/software-developer-wpf-at-permanent-personnel-services-pte-ltd-2209829667?refId=7f0a7435-aad9-4644-abe5-85a3fe500855&position=1&pageNum=38&trk=public_jobs_job-result-card_result-card_full-click
Software Developer (WPF)
PERMANENT PERSONNEL SERVICES PTE LTD
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 922 of 968
https://sg.linkedin.com/jobs/view/senior-software-developer-in-test-at-traveloka-technology-pte-ltd-2218798992?refId=7f0a7435-aad9-4644-abe5-85a3fe500855&position=2&pageNum=38&trk=public_jobs_job-result-card_result-card_full-click
Senior Software Developer in Test
TRAVELOKA TECHNO

https://sg.linkedin.com/jobs/view/software-engineer-pmet-at-st-electronics-training-simulation-systems-pte-ltd-2225288274?refId=7f0a7435-aad9-4644-abe5-85a3fe500855&position=22&pageNum=38&trk=public_jobs_job-result-card_result-card_full-click
Software Engineer - PMET
ST ELECTRONICS (TRAINING & SIMULATION SYSTEMS) PTE. LTD.
Singapore| Singapore
4 days ago
Be among the first 25 applicants


now at job number: 943 of 968
https://sg.linkedin.com/jobs/view/your-first-job-as-a-software-developer-at-beamandgo-pte-ltd-2220877924?refId=7f0a7435-aad9-4644-abe5-85a3fe500855&position=23&pageNum=38&trk=public_jobs_job-result-card_result-card_full-click
Your first job as a software developer
BeamAndGo Pte Ltd
Singapore| Singapore
1 week ago
Be among the first 25 applicants


now at job number: 944 of 968
https://sg.linkedin.com/jobs/view/software-developer-at-allforu-pte-ltd-2228174215?refId=658c729c-61e5-4985-b042-13106317e449&position=1&pageNum=39&trk=public_jobs_job-result-card_result-card_full-c

https://sg.linkedin.com/jobs/view/software-developer-at-anchor-global-consulting-pte-ltd-2233130533?refId=658c729c-61e5-4985-b042-13106317e449&position=22&pageNum=39&trk=public_jobs_job-result-card_result-card_full-click
Software Developer
ANCHOR GLOBAL CONSULTING PTE. LTD.
Singapore| Singapore
2 days ago
Be among the first 25 applicants


now at job number: 966 of 968
https://sg.linkedin.com/jobs/view/software-engineer-devops-at-garuda-robotics-pte-ltd-2225332059?refId=658c729c-61e5-4985-b042-13106317e449&position=23&pageNum=39&trk=public_jobs_job-result-card_result-card_full-click
Software Engineer (DevOps)
GARUDA ROBOTICS PTE. LTD.
Singapore| Singapore
2 weeks ago
Be among the first 25 applicants


now at job number: 967 of 968
https://sg.linkedin.com/jobs/view/lead-software-engineer-senior-software-engineer-software-engineer-at-golden-bridge-drive-sg-2220937312?refId=658c729c-61e5-4985-b042-13106317e449&position=24&pageNum=39&trk=public_jobs_job-result-card_result-card_full-click
L